# Stock-trading RL Agent (Adilbai) + NSE integration
Run cells step‑by‑step.

In [1]:
!git clone https://huggingface.co/Adilbai/stock-trading-rl-agent.git repo || true
!pip install -q stable-baselines3 huggingface_hub gym pandas numpy matplotlib ta nsepython nsetools

Cloning into 'repo'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 80 (delta 24), reused 0 (delta 0), pack-reused 6 (from 1)
Unpacking objects: 100% (80/80), 42.96 KiB | 1.65 MiB/s, done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 6.0 MB/s eta 0:00:00


In [8]:
with open('dataprocessor.py', 'r') as f:
    print(f.read())

import yfinance as yf
import pandas as pd
import numpy as np
from typing import List, Dict, Optional, Tuple
import os
import logging
from datetime import datetime, timedelta
import pickle
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class StockDataProcessor:
    """
    A comprehensive class for downloading, processing, and preprocessing stock data
    from Yahoo Finance for reinforcement learning applications.
    """
    
    def __init__(self, data_dir: str = "stock_data", cache_dir: str = "cache"):
        self.data_dir = data_dir
        self.cache_dir = cache_dir
        self.scalers = {}
        
        # Create directories if they don't exist
        os.makedirs(data_dir, exist_ok=True)
   

In [17]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

ticker = "RELIANCE.NS" # Using the yfinance symbol for Reliance (NSE)

# Calculate start and end dates for a longer period, e.g., 2 years
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365) # Fetch data for the last 2 years

# Fetch historical data from Yahoo Finance for the NSE-listed stock
data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

# Display the head of the DataFrame
display(data.head())

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2023-11-13,1148.797241,1153.760506,1147.357822,1152.916654,3854810
2023-11-15,1169.568481,1172.298277,1154.951691,1161.403935,12220648
2023-11-16,1171.677856,1178.353423,1164.853368,1166.913195,13134910
2023-11-17,1169.121826,1177.906780,1167.384684,1167.806488,7758634
2023-11-20,1166.044678,1170.536335,1159.617153,1165.647592,4490186


In [18]:
from dataprocessor import StockDataProcessor
import pandas as pd # Ensure pandas is imported for MultiIndex handling

# Ensure 'Date' is a column and add 'Ticker' column for the processor
data_processed = data.reset_index()
data_processed['Date'] = pd.to_datetime(data_processed['Date'])
data_processed['Ticker'] = ticker

# Flatten MultiIndex columns if they exist
if isinstance(data_processed.columns, pd.MultiIndex):
    # This assumes the relevant column names ('Close', 'Open', etc.) are in the first level
    # and the ticker name is in the second level (e.g., ('Close', 'RELIANCE.NS'))
    data_processed.columns = [col[0] if isinstance(col, tuple) else col for col in data_processed.columns]

# After flattening, the column from reset_index() might be named 'index', so rename it to 'Date'
data_processed = data_processed.rename(columns={'index': 'Date'})

# Initialize the StockDataProcessor
processor = StockDataProcessor()

# Apply the processing pipeline steps manually
print("Calculating technical indicators...")
data_with_indicators = processor.calculate_technical_indicators(data_processed)

print("Creating lagged features...")
data_with_lags = processor.create_lagged_features(data_with_indicators)

print("Creating future returns...")
data_with_targets = processor.create_future_returns(data_with_lags)

print("Cleaning and normalizing data...")
cleaned_data = processor.clean_and_normalize_data(data_with_targets)

print("Creating RL states and actions...")
rl_data, data_scaler = processor.create_rl_states_actions(cleaned_data)

print("Data preparation for RL observation complete.")

# The 'rl_data' dictionary now contains the states for each ticker.
# For 'RELIANCE.NS', the states would be in rl_data['RELIANCE.NS']['states']
print(f"RL data prepared for {len(rl_data)} stock(s).")
if ticker in rl_data:
    reliance_states = rl_data[ticker]['states']
    print(f"{ticker} states shape: {reliance_states.shape}")
    print(f"Example of a single observation (last state in sequence):")
    # Display the last observation as an example
    if reliance_states.shape[0] > 0:
        print(reliance_states[-1])
    else:
        print("No states generated for this ticker, possibly due to insufficient data after processing.")

# The 'data_scaler' is the scaler used for normalization, similar to the loaded 'scaler' from the model files.


Calculating technical indicators...
Creating lagged features...
Creating future returns...
Cleaning and normalizing data...
Creating RL states and actions...
Data preparation for RL observation complete.
RL data prepared for 1 stock(s).
RELIANCE.NS states shape: (384, 60, 50)
Example of a single observation (last state in sequence):
[[-0.07972499 -0.16294573 -0.06449771 ...  0.53152966  0.89285697
   1.56205419]
 [-0.13241723 -0.21953975 -0.08411825 ...  0.36965828  0.6276044
   1.65196096]
 [-0.00319202 -0.07509123 -0.01361813 ...  0.14745096  0.53699577
   1.6200796 ]
 ...
 [ 0.95328169  1.01030108  0.9999447  ...  1.42568066  1.39687368
   0.12852526]
 [ 1.06113912  0.97774997  0.99688024 ...  1.36577368  1.4576751
   0.32477037]
 [ 0.8912129   1.0041979   0.99279304 ...  1.31262366  1.43400306
   0.68564697]]


In [ ]:
# Get the last observation from the processed data
# The model expects a single observation of shape (1, sequence_length, n_features)
current_observation = reliance_states[-1]

# Reshape for prediction: add a batch dimension
obs_for_prediction = np.expand_dims(current_observation, axis=0)

print(f"Shape of observation for prediction: {obs_for_prediction.shape}")

# Get trading decision from the model
action, _states = model.predict(obs_for_prediction, deterministic=True)

# Interpret action
action_type = ["HOLD", "BUY", "SELL"][int(action[0])]
position_size = action[1]     # 0-1: Fraction of available capital

print(f"\nModel's Suggested Action: {action_type}, Position Size: {position_size:.2%}")


Now that we have the raw historical data, the next step is to process it into the correct observation format that the model expects. The `README.md` mentions a `prepare_observation` function and suggests using `dataprocessor.py` for data preparation. We will need to investigate `dataprocessor.py` to understand how to create this function.

In [6]:
from stable_baselines3 import PPO
import pickle
import numpy as np

# Load the trained model
model = PPO.load("final_model")

# Load the data scaler
with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

print("Model and scaler loaded successfully!")

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/save_util.py:165: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  deserialized_object = cloudpickle.loads(base64_object)


Model and scaler loaded successfully!


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
with open('README.md', 'r') as f:
    print(f.read())

---
library_name: stable-baselines3
tags:
- reinforcement-learning
- trading
- finance
- stock-market
- ppo
- quantitative-finance
- algorithmic-trading
- deep-reinforcement-learning
- portfolio-management
- financial-ai
license: mit
base_model: PPO
model-index:
- name: Stock Trading RL Agent
  results:
  - task:
      type: reinforcement-learning
      name: Stock Trading
    dataset:
      name: FAANG Stocks (5Y Historical Data)
      type: financial-time-series
    metrics:
    - type: total_return
      value: 162.87
      name: Best Total Return (AMZN)
    - type: sharpe_ratio
      value: 0.74
      name: Best Sharpe Ratio (AMZN)
    - type: max_drawdown
      value: 145.29
      name: Best Max Drawdown (TSLA)
    - type: win_rate
      value: 52.11
      name: Best Win Rate (MSFT)
datasets:
- yahoo-finance
pipeline_tag: reinforcement-learning
widget:
- text: "Technical Analysis Trading Agent"
  example_title: "Stock Trading Decision"
---

# 🚀 Stock Trading RL Agent - Advanced PP

In [2]:
import os
os.chdir('repo')
print(os.listdir('.'))

['eval_MSFT.monitor.csv', 'scaler.pkl', '.gitattributes', 'eval_TSLA.monitor.csv', 'train_TSLA.monitor.csv', 'eval_AAPL.monitor.csv', 'AAPL_performance.png', 'enviromentcreator.py', 'resumer_MyTestExp_20250704_171133.log', 'train_GOOGL.monitor.csv', 'trainer_MyTestExp_20250704_165810.log', 'trainer_MyTestExp_20250704_171341.log', 'train_MSFT.monitor.csv', 'GOOGL_performance.png', 'resumer_MyTestExp_20250704_170925.log', 'trainer_MyTestExp_model_info.json', 'resumer_MyTestExp_20250704_171431.log', 'trainer_MyTestExp_20250704_171133.log', 'README.md', 'config.json', 'processed_data.pkl', '.git', 'resumer_MyTestExp_20250704_171341.log', 'tensorboard', 'train_AMZN.monitor.csv', 'trainer_MyTestExp_data_info.json', 'eval_AMZN.monitor.csv', 'MSFT_performance.png', 'eval_GOOGL.monitor.csv', 'evaluation_results.json', 'trainer_MyTestExp_20250704_170925.log', 'train_AAPL.monitor.csv', 'trainer_MyTestExp_20250704_171431.log', 'trainer_MyTestExp_20250704_164102.log', 'dataprocessor.py', 'final_mod

This will change your current working directory to `repo` and list its contents. Look for a main script (e.g., `main.py`, `train.py`, or `run.py`) or another Jupyter Notebook that might serve as the entry point for the RL agent.